In [1]:
import nltk
import pandas as pd
import sys
import os
import io
import operator

from data_loader import DataLoader

from nltk.tokenize import word_tokenize ,sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer
from nltk.cluster.util import cosine_distance
from nltk import sent_tokenize, word_tokenize, PorterStemmer
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from nltk import pos_tag

import math

import numpy

import spacy
from spacy.lang.en.stop_words import STOP_WORDS

import networkx as nx

import re

import networkx as nx

from collections import defaultdict
import string

import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

import copy
import itertools

import pke

from  itertools import chain

In [2]:
loader = DataLoader(DataLoader.data_path1)
table = loader.load_table()
table.head()

C:\Users\lijia\Desktop\MSAI\Sem A\AI6122\Assignment1\Hearts-of-Iron\data_loader.py:13: FutureWarning: Starting with pandas version 2.0 all arguments of read_json except for the argument 'path_or_buf' will be keyword-only.
  self.table = pd.read_json(data_path, 'records', lines = True);


,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,AIVZMEFEAGF3,B0090UTDKO,txlucky1,"[2, 2]",Although it would seem that most of the inform...,5,Surprisingly useful advice,1367280000,"04 30, 2013"
1,A17O3IALB0VVSA,B00ASFHQUY,mapleleafmanny,"[2, 2]",I've always loved the story of Peter Pan and I...,4,Better then the classic story,1379635200,"09 20, 2013"
2,A1Q6090OWANRMC,B00B6EFPEK,Bev Seabrook,"[18, 19]",Most people equate being frugal with being dep...,5,Start Living a Better Life Today,1359590400,"01 31, 2013"
3,A171L7UJTO7HXU,B00AQJRYWC,Ro13135,"[0, 0]","Good read, funny at times but author wrapped u...",4,Good Book but disapointing ending,1405728000,"07 19, 2014"
4,A2ISDBOVMRCRLO,B00ARY7ZUC,"C. L. Orlando ""always reading""","[0, 0]",I like the realistic viewpoints of the heroine...,4,Liking this series very much,1373846400,"07 15, 2013"


In [3]:
#Data Preprocess
#Get stopwords
nltk_stopwords = stopwords.words('english')
nltk_stopwords.append('\n')
spacy_stopwords = list(STOP_WORDS)
merged_sw = nltk_stopwords + list(set(spacy_stopwords) - set(nltk_stopwords))

print("nltk: ", len(nltk_stopwords))
print("spacy: ", len(spacy_stopwords))
print("merge: ", len(merged_sw))
print(merged_sw)

nltk:  180
spacy:  326
merge:  383
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'on

In [4]:
#Create the review text dictionary
reviewText = {}
for i in range(200):
    if table['asin'][i] in reviewText:
        chunk = table['reviewText'][i]
        sentences = nltk.sent_tokenize(chunk)
        for s in sentences:
            reviewText[table['asin'][i]].append(s)
    else:
        reviewText[table['asin'][i]] = []
        chunk = table['reviewText'][i]
        sentences = nltk.sent_tokenize(chunk)
        for s in sentences:
            reviewText[table['asin'][i]].append(s)
print(reviewText)

{'B0090UTDKO': ["Although it would seem that most of the information is common sense, you'd be surprised at the number of people who make mistakes warned about in this book.", 'It may be too simply for the seasoned investor, but for everyone else, it is worth the quick read.', "May I recommend Mr. Gallinelli's book What every real estate investor needs to know about cash flow.... for those who wish to get into the nitty gritty of real estate investing mathematics."], 'B00ASFHQUY': ["I've always loved the story of Peter Pan and I think the author did a fantastic job of capturing the essence of the story and Tinkerbell's personality in this modern rendition.This story actually makes more sense then the fairytale version because it ties off loose ends, such as why Hook is there, how Peter Pan got there, how the Lost Boys arrived, even the addition of more fairies then just Tinkerbell makes more sense.", 'Now THIS is a book series that would make a wonderful retelling of Peter Pan.The atmo

In [5]:
def data_preprocess(original_review, stop_words):
    # to lowercase
    review = original_review.lower()
    # remove punctuation
    review = re.sub(r'[^\w\s]', ' ', review).strip()

    lemmatizer = WordNetLemmatizer()
    review_words = review.split(' ')
    review_words_tag = nltk.pos_tag(review_words)
    processed_review = ""
    # remove stop words&lemma
    for word, tag in review_words_tag:
        wntag = tag[0].lower()
        wntag = wntag if wntag in ['a', 'r', 'n', 'v'] else None
        if not wntag:
            lemma_word = word
        else:
            lemma_word = lemmatizer.lemmatize(word, wntag)
        if lemma_word not in stop_words:
            processed_review = processed_review + lemma_word + " "
    processed_review = " ".join(processed_review.split())
    processed_review = processed_review.strip()
    return processed_review



In [6]:
processed_reviewText = {}
for r in reviewText:
    processed_reviewText[r] = []
    for t in reviewText[r]:
        p = data_preprocess(t, merged_sw)
        processed_reviewText[r].append(p)
print(processed_reviewText)

{'B0090UTDKO': ['information common sense surprise number people mistake warn book', 'simply seasoned investor worth quick read', 'recommend mr gallinelli book real estate investor need know cash flow wish nitty gritty real estate invest mathematics'], 'B00ASFHQUY': ['love story peter pan think author fantastic job capture essence story tinkerbell personality modern rendition story actually sense fairytale version tie loose end hook peter pan lost boy arrive addition fairy tinkerbell sense', 'book series wonderful retelling peter pan atmosphere beautifully build island sound gorgeous', 'character development spot accurate', 'exaggerated', 'storyline flow beautifully installment interesting want read'], 'B00B6EFPEK': ['people equate frugal deprive', 'book provide countless specific suggestion save money area life', 'learn trim food expense clothe utility travel expense holiday gift home appliance pet care entertainment', 'tip save money buy musical instrument kid love book drastically c

In [7]:
# collect frequency

def collect_frequency(processed_review):
    word_frequency = defaultdict(lambda:0)
    for v in processed_review.values():
        for i in v:
            tokens = nltk.word_tokenize(i)
            for token in tokens:
                word_frequency[token] += 1  
    return word_frequency

frequency = collect_frequency(processed_reviewText)

In [8]:
frequency = sorted(frequency.items(), key=lambda item: item[1],reverse=True)
print(frequency)

[('book', 340), ('story', 202), ('read', 194), ('love', 159), ('like', 109), ('character', 82), ('author', 75), ('good', 68), ('8217', 68), ('want', 67), ('end', 65), ('way', 62), ('time', 61), ('series', 60), ('great', 60), ('think', 57), ('life', 55), ('find', 53), ('know', 52), ('write', 46), ('enjoy', 45), ('34', 43), ('thing', 40), ('start', 39), ('come', 38), ('lot', 37), ('romance', 36), ('recommend', 34), ('world', 32), ('wait', 31), ('feel', 30), ('work', 29), ('review', 29), ('short', 29), ('people', 28), ('woman', 28), ('reader', 27), ('need', 26), ('definitely', 26), ('family', 26), ('look', 25), ('little', 25), ('new', 25), ('real', 24), ('right', 24), ('leave', 24), ('away', 24), ('help', 24), ('happen', 24), ('hot', 23), ('year', 23), ('best', 23), ('heart', 23), ('friend', 23), ('day', 22), ('past', 22), ('star', 21), ('hard', 21), ('hope', 21), ('black', 21), ('sure', 20), ('try', 20), ('let', 20), ('main', 19), ('girl', 19), ('buy', 18), ('5', 18), ('point', 18), ('ba

In [9]:
sorted_processed_reviewText = sorted(processed_reviewText.items(), key=lambda item: len(item[1]),reverse=True)
print(sorted_processed_reviewText)

[('B00GVZZQ2A', ['absolutely love monica decide write novella drew fable', 'absolute perfection guy', 'seriously', 'love endure deserve happiness', 'easy road', 'fight future watch earn hea short beautiful 34 come run away face fear', '', 'struggle good right cherish', 'hold close nurture protect sure let guard', 'feel husband 34 e arc 10 fable draw pressure drew nfl career consider', 'fable feisty strong character insecurity', 'afraid ifs drew find away', '', 'handle distance', 'fault think way', 'know drew fine specimen let forget utter love', 'case point 34 woman', 'longer girl anymore', 'jaded angry defiant girl determined fight world shade shine', 'love girl', 'fiercely', 'love woman fiercely', 'fianc eacute e future wife future mother child', 'fable woman stand', 'forever 34 e arc 9 holy heck', 'drew callahan real effing deal folk', 'rough stuff emerge victorious', 'work rebuild relationship father form pretty special fable younger brother owen', 'like owen look draw drew great r

In [10]:
# #DM B00004YKUI 63> B000002NBY 48 > B000F0UV3Q 37
# print(len(processed_reviewText['B000002NBY']))

In [11]:
#KS B00GVZZQ2A 30> B00LKNP4T8 30 > B00EAZEQN8 26 > B00H0ESPII 26
print(len(processed_reviewText['B00GVZZQ2A']))

30


In [12]:
# Abstractive Summarizer by Model
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
import torch

model_name = 'google/pegasus-xsum'
torch_device = 'cuda'
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(torch_device)

text = """"""
for i in reviewText['B00GVZZQ2A']:
    text+=i
print(text)

batch = tokenizer.prepare_seq2seq_batch(text, truncation=True, padding='longest',return_tensors='pt').to(torch_device)
translated = model.generate(**batch)
tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)

print(tgt_text)

I absolutely LOVED that Monica decided to write this novella for Drew and Fable!It was absolute perfection you guys.Seriously.I love these two so much and after everything they've both endured, they deserve to have their happiness.It hasn't been an easy road and we really see that here.These two have had to fight for their future together and watching them earn their HEA is nothing short of beautiful.&#34;We came together, he ran away, and then we faced our fears.Together.When you struggle for something so good, so right, you cherish it more.You hold it close, nurture it, protect it, make sure you never, ever let your guard down.That's how I feel about my almost husband.&#34; (e-ARC 10%)Fable and Drew now have the pressures of Drew's NFL career to consider.Fable, while a feisty, strong character, has her insecurities.She is afraid of all the what ifs- what if Drew finds someone better while he's away?What if she's not really the one for him?What if she can't handle the distance?I can't

C:\Users\lijia\anaconda3\lib\site-packages\transformers\tokenization_utils_base.py:3415: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and the tokenizer under the `as_target_tokenizer` context manager to prepare
your targets.

Here is a short example:

model_inputs = tokenizer(src_texts, ...)
with tokenizer.as_target_tokenizer():
    labels = tokenizer(tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)


["It's been a long time coming, but it's finally here!"]


In [13]:
#TextRank
# define the set of valid Part-of-Speeches
pos = {'NOUN', 'PROPN', 'ADJ'}

# 1. create a TextRank extractor.
extractor = pke.unsupervised.TextRank()

# 2. load the content of the document.
extractor.load_document(input=text,
                        language='en',
                        normalization=None)

# 3. build the graph representation of the document and rank the words.
#    Keyphrase candidates are composed from the 33-percent
#    highest-ranked words.
extractor.candidate_weighting(window=2,
                              pos=pos,
                              top_percent=0.33)

# 4. get the 10-highest scored candidates as keyphrases
keyphrases = extractor.get_n_best(n=10)

keyphrases

[('effing deal', 0.05873795740158209),
 ("sf 49'ers", 0.04524668012022184),
 ('married couple', 0.04524653012022184),
 ('younger brother', 0.04524597012022184),
 ('rough stuff', 0.04524568012022184),
 ('defiant girl', 0.04524478012022184),
 ('utter love', 0.04524440012022184),
 ('fine specimen', 0.04524431012022184),
 ('strong character', 0.04524375012022184),
 ('nfl career', 0.04524364012022184)]

In [14]:
# 1. create a YAKE extractor.
extractor = pke.unsupervised.YAKE()

# 2. load the content of the document.
extractor.load_document(input=text,
                        language='en',
                        normalization=None,
                        stoplist=merged_sw)


# 3. select {1-3}-grams not containing punctuation marks and not
#    beginning/ending with a stopword as candidates.
extractor.candidate_selection(n=3)

# 4. weight the candidates using YAKE weighting scheme, a window (in
#    words) for computing left/right contexts can be specified.
window = 2
use_stems = False # use stems instead of words for weighting
extractor.candidate_weighting(window=window,
                              use_stems=use_stems)

# 5. get the 10-highest scored candidates as keyphrases.
#    redundant keyphrases are removed from the output using levenshtein
#    distance and a threshold.
threshold = 0.8
keyphrases = extractor.get_n_best(n=10, threshold=threshold)

keyphrases

[('monica decided', 0.022761686886864425),
 ('drew', 0.027816791172470524),
 ('monica', 0.07540552328123676),
 ('fable', 0.08286031730719266),
 ('absolute perfection', 0.09541877870830255),
 ('owen', 0.13773357011159784),
 ('love', 0.15573782440496314),
 ('absolutely loved', 0.16187732218749834),
 ('future', 0.21906331755251804),
 ('girl', 0.22009540189435522)]

In [15]:
# 1. create a TfIdf extractor.
extractor = pke.unsupervised.TfIdf()

# 2. load the content of the document.
stoplist = list(string.punctuation)
stoplist += pke.lang.stopwords.get('en')
extractor.load_document(input=text,
                        language='en',
                        stoplist=merged_sw,
                        normalization=None)

# 3. select {1-3}-grams not containing punctuation marks as candidates.
extractor.candidate_selection(n=3)

# 4. weight the candidates using a `tf` x `idf`
df = pke.load_document_frequency_file(input_file='df.tsv.gz')
extractor.candidate_weighting(df=df)

# 5. get the 10-highest scored candidates as keyphrases
keyphrases = extractor.get_n_best(n=10)

keyphrases

[('drew', 15.849625007211563),
 ('fable', 7.924812503605781),
 ('love', 6.339850002884625),
 ('future', 4.754887502163469),
 ('woman', 4.754887502163469),
 ('girl', 4.754887502163469),
 ('owen', 4.754887502163469),
 ('loved', 3.1699250014423126),
 ('fight', 3.1699250014423126),
 ('let', 3.1699250014423126)]

In [16]:
# TopicRank
# initialize keyphrase extraction model, here TopicRank
extractor = pke.unsupervised.TopicRank()

# load text
extractor.load_document(input=text, language='en')

# keyphrase candidate selection, in the case of TopicRank: sequences of nouns
# and adjectives (i.e. `(Noun|Adj)*`)
extractor.candidate_selection()

# candidate weighting, in the case of TopicRank: using a random walk algorithm
extractor.candidate_weighting()

# N-best selection, keyphrases contains the 10 highest scored candidates as
# (keyphrase, score) tuples
keyphrases = extractor.get_n_best(n=10)

keyphrases

[('drew', 0.1201836597302716),
 ('fable', 0.07570900388688667),
 ('owen', 0.0422845464761035),
 ('woman', 0.031769340276900475),
 ('girl', 0.0301432913584528),
 ('future', 0.029415923166387088),
 ('e-arc', 0.023724469370331583),
 ('utter love', 0.021658123529877324),
 ('years', 0.01951480454956794),
 ('younger brother', 0.019431710401101282)]

In [17]:
# RAKE Algorithm
def is_number(s):
    try:
        float(s) if '.' in s else int(s)
        return True
    except ValueError:
        return False

def load_stop_words(stop_word_file, regex):
    with io.open(stop_word_file, encoding='utf8') as stop_word_file:
        stop_words = re.split(regex, stop_word_file.read())
    return [word for word in stop_words if word not in ('', ' ')]  # filters empty string matches


def separate_words(text):
    """
    Utility function to return a list of all words that are have a length greater than a specified number of characters.
    @param text The text that must be split in to words.
    @param min_word_return_size The minimum no of characters a word must have to be included.
    """
    splitter = re.compile(r'(?u)\W+')
    words = []
    for single_word in splitter.split(text):
        current_word = single_word.strip().lower()
        # leave numbers in phrase, but don't count as words, since they tend to invalidate scores of their phrases
        if current_word != '' and not is_number(current_word):
            words.append(current_word)
    return words


def split_sentences(text):
    """
    Utility function to return a list of sentences.
    @param text The text that must be split in to sentences.
    """
    sentence_delimiters = re.compile(u'[.!?,;:\t\\\\"\\(\\)\\\'\u2019\u2013]|\\s\\-\\s')
    sentences = sentence_delimiters.split(text)
    return sentences


def build_stop_word_regex(stop_word_list):
    stop_word_regex_list = []
    for word in stop_word_list:
        word_regex = r'\b' + word + r'(?![\w-])'
        stop_word_regex_list.append(word_regex)
    return re.compile('(?u)' + '|'.join(stop_word_regex_list), re.IGNORECASE)


def generate_candidate_keywords(sentence_list, stop_word_pattern, minCharacters, maxWords):
    phrase_list = []
    for s in sentence_list:
        tmp = re.sub(stop_word_pattern, '|', s.strip())
        phrases = tmp.split("|")
        for phrase in phrases:
            phrase = phrase.strip().lower()
            if phrase != '' and len(phrase) >= minCharacters and len(phrase.split()) <= maxWords:
                phrase_list.append(phrase)
    return phrase_list


def calculate_word_scores(phraseList):
    word_frequency = {}
    word_degree = {}
    for phrase in phraseList:
        word_list = separate_words(phrase)
        word_list_length = len(word_list)
        word_list_degree = word_list_length - 1
        for word in word_list:
            word_frequency.setdefault(word, 0)
            word_frequency[word] += 1
            word_degree.setdefault(word, 0)
            word_degree[word] += word_list_degree
    for item in word_frequency:
        word_degree[item] = word_degree[item] + word_frequency[item]

    # Calculate Word scores = deg(w)/frew(w)
    word_score = {}
    for item in word_frequency:
        word_score.setdefault(item, 0)
        word_score[item] = word_degree[item] / (word_frequency[item] * 1.0)
    return word_score


def generate_candidate_keyword_scores(phrase_list, word_score, minFrequency):
    keyword_candidates = {}
    # iterate over the list once to count how many times
    # each phrase occurs
    phrase_counts = defaultdict(int)
    for p in phrase_list:
        phrase_counts[p] += 1
    # and then a second time to compute RAKE scores
    for phrase in phrase_list:
        if phrase_counts[phrase] >= minFrequency:
            keyword_candidates.setdefault(phrase, 0)
            word_list = separate_words(phrase)
            candidate_score = 0
            for word in word_list:
                candidate_score += word_score[word]
            keyword_candidates[phrase] = candidate_score
    return keyword_candidates

def sort_tuple(tup):
    tup.sort(key = lambda x:x[1])
    return tup

class Rake(object):

    def __init__(self, stop_words, regex=r'[\W\n]+'):
        #lets users call predefined stopwords easily in a platform agnostic manner or use their own list
        if isinstance(stop_words, list):
            self.__stop_words_pattern = build_stop_word_regex(stop_words)
        else:
            self.__stop_words_pattern = build_stop_word_regex(load_stop_words(stop_words, regex))

    def run(self, text, minCharacters=1, maxWords=5, minFrequency=1):
        sentence_list = split_sentences(text)

        phrase_list = generate_candidate_keywords(sentence_list, self.__stop_words_pattern, minCharacters, maxWords)

        word_scores = calculate_word_scores(phrase_list)

        keyword_candidates = generate_candidate_keyword_scores(phrase_list, word_scores, minFrequency)

        sorted_keywords = sorted(keyword_candidates.items(), key=operator.itemgetter(1), reverse=True)
        return sorted_keywords

In [18]:
rake_object = Rake(merged_sw)

keywords = sort_tuple(rake_object.run(text))
print(keywords[len(keywords)-int(math.sqrt(len(keywords))):])

[('emerged victorious', 4.0), ('pretty special', 4.0), ('younger brother', 4.0), ('looking forward', 4.0), ('years later', 4.0), ('married couple', 4.0), ('rock solid', 4.0), ('great role model', 9.0), ('high profile job', 9.0), ('real effing deal folks', 16.0)]
